# Fitting a GMM to the AdVitam Dataset


<hr style="border-top: 1px solid white;">


## Preamble


Python Libraries


In [1]:
import numpy as np
import pandas as pd
import neurokit2 as nk
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.mixture import GaussianMixture
from sklearn.model_selection import (
    train_test_split,
    GridSearchCV,
    RandomizedSearchCV
)

<br></br>


Custom Functions


In [2]:
# in order of appearance
from useful_functions.check_for_missing_data import check_for_missing_data

# driving data
from useful_functions.driving_data.dd_dictionary import create_dd_dictionary
from useful_functions.driving_data.process_driving_data import processing_driving_data
from useful_functions.takeover_dataframe import create_takeover_timestamps

# physio data
from useful_functions.physio_data.pd_dictionary import create_pd_dictionary
from useful_functions.physio_data.process_physio_timestamps import process_physio_timestamps
from useful_functions.physio_data.process_physio_data import process_physio_data

from useful_functions.demographic_data.process_driver_demographic_data import (
    process_driver_demographic_data,
)

from useful_functions.construct_observations import construct_observations

<br></br>

Storing the folder paths to raw data


In [3]:
driving_data_folder = "../AdVitam/Exp2/Raw/Driving"
physio_data_folder = "../AdVitam/Exp2/Raw/Physio/Txt"

<br></br>

Storing a list of driver files to exclude

In [4]:
drivers_to_exclude = check_for_missing_data(driving_data_folder, physio_data_folder)
drivers_to_exclude.extend(["NST77", "NST11", "ST22", "NST87", "ST14", "ST12", "NST73", "ST10"])

`check_for_missing_data()` returns a list of any driver that is _not_ in both the `driving` and `physio` folders.

Participants: ST22, NST87, ST14, ST12, NST73 and ST10 seem to have issues with there physiological data

<br></br>
<br></br>

# Importing Data + Preprocessing


---


<br>


## Driving Data


<hr style="border-top: 1px dashed white; border-bottom: 0px">


### Data Description


**Driver Data:**
| Feature | Description | Notes |
| --- | --- | --- |
| Time | Time elapsed since the software was launched (in seconds) | |
| EngineSpeed | Engine speed (in rpm) | Removed |
| GearPosActual | Current gear | Removed |
| GearPosTarget | Next planned gear | Removed |
| AcceleratorPedalPos | Position of gas pedal. | Recording problem, Removed |
| DeceleratorPedalPos | Position of brake pedal. | Recording problem, Removed |
| SteeringWheelAngle | Steering wheel angle (in degrees) | |
| VehicleSpeed | Vehicle speed (in mph) | |
| Position X | Vehicle position along the x-axis in the simulated driving environment | |
| Position Y | Vehicle position along the y-axis in the simulated driving environment | |
| Position Z | Vehicle position along the z-axis in the simulated driving environment | |
| Autonomous Mode (T/F) | Autonomous pilot status. The driver is in control of the car when the value of the column "Autonomous Mode (T/F)" is False | True = Activated, False = Deactivated (driver in control) |
| Obstacles | Events that occurred during the driving simulation. | See Below |


<br></br>


**Obstacles:**

| Event         | Description                                                                                                                                                                                             |
| ------------- | ------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- |
| TriggeredObsX | Time at which each takeover request was triggered by the experimenter                                                                                                                                   |
| Obs1          | Deer                                                                                                                                                                                                    |
| Obs2          | Traffic cone                                                                                                                                                                                            |
| Obs3          | Frog                                                                                                                                                                                                    |
| Obs4          | Traffic cone                                                                                                                                                                                            |
| Obs5          | False alarm (x2)                                                                                                                                                                                        |
| Detected      | Time at which the driver pressed the steering wheel button to notify he/she understood the situation. |


<br></br>


### Driving Data Dictionary


Creates a dictionary of the raw driving data files.


In [5]:
driving_data_dictionary = create_dd_dictionary(driving_data_folder, drivers_to_exclude)

`create_dd_dictionary()` stores every file in the `driving_data_folder` in a dictionary

<br></br>


Processing driving data


In [6]:
# Fitting a Label Encoder to the `Obstacles` column
driver_data = driving_data_dictionary["NST01"]
driver_data = driver_data.fillna("Nothing")
enc = preprocessing.LabelEncoder()
enc.fit(driver_data["Obstacles"])

# Processing the driving data
driving_data_dictionary = processing_driving_data(driving_data_dictionary, enc)

A label Encoder is fit to the `Obstacles` of driver `NST01`

`processing_driving_data()`label encodes the `Obstacles` for every driver and resamples the driving data to 10ms (or 100Hz)

<br></br>


Creating driving data takeover timestamps


In [7]:
driving_timestamps = create_takeover_timestamps(driving_data_dictionary, enc)

`create_takeover_timestamps()` extracts timestamps from each driver takeover.

Driving Timestamps include:
- `TriggeredObsX` : When the takeover request for obstacle X was triggered
- `TakeoverObsX`: When the driver tookover the vehicle
- `ReleaseObsX`: When the driver released control of the vehicle to the automation
- `TOTObsX`: The amount of time it took for the driver to takeover for obstacle X (`TakeoverObsX` - `TriggeredObsX`)


<br></br>
<br></br>


## Physiological Signals & Markers


<hr style="border-top: 1px dashed white; border-bottom: 0px">


### Data Description


**Signals:**

| Feature | Description            | Notes  |
| ------- | ---------------------- | ------ |
| min     | Time Elapsed           |        |
| ECG     | Electrocardiogram      | 1000Hz |
| EDA     | Electrodermal Activity | 1000Hz |
| RESP    | Resperatory            | 1000Hz |


<br></br>


**Markers:**

Contains the timestamps for each period of the experiment.

- Training1 = Baseline phase
- Training2 = Practice phase in the driving simulator
- Driving = Main driving session in conditionally automated driving.

Be careful, the timestamps are here in seconds while they are in minutes in the raw data.


<br></br>


**Timestamps:**

Time elapsed (in seconds) between the start of the main driving session and the appearance of the obstacles.

- TrigObsX: the time when the driver pressed the button to report having understood the situation
- DetObsX: and the time when the driver actually took over control
- RepObsX: X corresponds to one of obstacle or the false alarm.


<br></br>


### Physio data dictionary


Creates a dictionary of the raw physiological data and their markers


In [8]:
phsyiological_data_dictionary = create_pd_dictionary(physio_data_folder, drivers_to_exclude)

`create_pd_dictionary()` stores every file in the `physio_data_folder` in a dictionary

<br></br>


Processing the Physiological data


In [9]:
phsyiological_data_dictionary = process_physio_data(phsyiological_data_dictionary)

`process_physio_data()` resamples the data to 10ms (100Hz) and then segments the data into each experimental phase (Baseline, Training, Driving)


<br></br>


### Physio timestamps


A dataframe to store the trigger time, takeover time, release time, and TOT for each obstacle, for every driver. Similar to `driver_timestamps`.


In [10]:
physio_timestamps = pd.read_csv(
    "../AdVitam/Exp2/Preprocessed/Physio and Driving/timestamps_obstacles.csv"
)

<br></br>


### Processing the Physio Timestamps

Steps:

1. Change column names to match driving timestamps
1. Remove preselected drivers
1. Reformat subject id to match
1. Transfrom timestamps into timedelta objects


In [11]:
physio_timestamps = process_physio_timestamps(physio_timestamps, drivers_to_exclude)

<br></br>


## Driver Demographic Data


<hr style="border-top: 1px dashed white; border-bottom: 0px">


### Data Description


**Driver Demographic Data Description:**
| Feature | Description | Note |
| --- | --- | --- |
| code | Code of driver Secondary Task (ST) vs No ST (NST) + unique id (1,2,...) | In the form (ST/NST)# |
| date | Day of data collection | Removed |
| time | Hour of data collection | Removed |
| condition | Experimental condition for mental workload | Removed (contained in driver code |
| sex | driver sex | |
| age | Age of drivers in years | |
| mothertongue | drivers first language | |
| education | Highest education degree | |
| driving_license | Year of obtenstion of driving license | |
| km_year | Number of kilometers covered per year in average | |
| accidents | Number of accidents during the last 3 years | |
| nasa_tlx_N | Answer to the NASA TLX for question N | Removed |
| danger_O | Subjective ranking of the danger of obstacle O | Removed |
| realism_O | Subjective ranking of the realism of obstacle O | Removed |
| sart_N_O | Subjective answer to the sart for question N related to obstacle O | Removed |
| demand_O | Demands on attentional resources (complexity, variability, and instability of the situation) | Removed |
| supply_O | Supply of attentional resources (division of attention, arousal, concentration, and spare mental capacity) | Removed |
| understanding_O | Understanding of the situation (information quantity, information quality and familiarity). | Removed |


<br></br>


### Driver Demographic Data


Grabbing the driver demographic data


In [12]:
driver_demographic_data = pd.read_csv(
    "../AdVitam/Exp2/Preprocessed/Questionnaires/Exp2_Database.csv",
    usecols=[
        "code",
        "sex",
        "age",
        "mothertongue",
        "education",
        "driving_license",
        "km_year",
        "accidents",
    ],
)

<br></br>


### Processing driver demographic data


Steps:

1. Remove preselected drivers
2. Reformat code to match data
3. Coverting driving licence from year obtained to of years obtained
4. Normalize km/y


In [13]:
driver_demographic_data = process_driver_demographic_data(
    driver_demographic_data, drivers_to_exclude
)

<br></br>


# Constructing Sequence of Observations


---


The idea is to train 2 HMM trained to observations assosiated with a 'slow' takeover, and a 'fast' takeover.


In [14]:
slow_observations, fast_observations, observations = construct_observations(
    driving_data_dictionary,
    phsyiological_data_dictionary,
    driving_timestamps,
    physio_timestamps,
    driver_demographic_data,
)

In [21]:
print(observations)

Index(['SteeringWheelAngle', 'VehicleSpeed', 'ECG_Raw', 'ECG_Clean',
       'ECG_Rate', 'ECG_Quality', 'ECG_R_Peaks', 'ECG_P_Peaks', 'ECG_P_Onsets',
       'ECG_P_Offsets', 'ECG_Q_Peaks', 'ECG_R_Onsets', 'ECG_R_Offsets',
       'ECG_S_Peaks', 'ECG_T_Peaks', 'ECG_T_Onsets', 'ECG_T_Offsets',
       'ECG_Phase_Atrial', 'ECG_Phase_Completion_Atrial',
       'ECG_Phase_Ventricular', 'ECG_Phase_Completion_Ventricular', 'RSP_Raw',
       'RSP_Clean', 'RSP_Amplitude', 'RSP_Rate', 'RSP_RVT', 'RSP_Phase',
       'RSP_Phase_Completion', 'RSP_Symmetry_PeakTrough',
       'RSP_Symmetry_RiseDecay', 'RSP_Peaks', 'RSP_Troughs', 'EDA_Raw',
       'EDA_Clean', 'EDA_Tonic', 'EDA_Phasic', 'SCR_Onsets', 'SCR_Peaks',
       'SCR_Height', 'SCR_Amplitude', 'SCR_RiseTime', 'SCR_Recovery',
       'SCR_RecoveryTime', 'RSA_P2T', 'RSA_Gates', 'code', 'sex', 'age',
       'mothertongue', 'education', 'driving_license', 'km_year', 'accidents',
       'NDRT'],
      dtype='object')


<br>
<br>


# Training the HMMs


---


**Train/Validate/Test Split**

In [16]:
slow_observations_train, slow_observations_test = train_test_split(
    slow_observations, test_size=0.1
)

fast_observations_train, fast_observations_test = train_test_split(
    fast_observations, test_size=0.1
)

<br>
<br>


# Hyperparameters


---


In [17]:
# initializing the hyperparameters
n_components = np.arange(1, 11)
covariance_type = ["full", "tied", "diag", "spherical"]
tol = np.arange(0.001, 0.011, 0.001)
init_params = ["kmeans", "k-means++", "random", "random_from_data"]
random_state = np.arange(0, 11)
max_iter = np.linspace(100, 10000, 100).astype(int)

hyperparametes = {
    "n_components": n_components,
    "covariance_type": covariance_type,
    "tol": tol,
    "init_params": init_params,
    "random_state": random_state,
    "max_iter": max_iter,
}

# initialize the model
slow_model = GaussianMixture(reg_covar=1e-4)
fast_model = GaussianMixture(reg_covar=1e-4)

# initialize the random search
slow_random_search = RandomizedSearchCV(
    slow_model, hyperparametes, n_iter=1000, cv=5, n_jobs=-1, error_score='raise'
)
fast_random_search = RandomizedSearchCV(
    fast_model, hyperparametes, n_iter=1000, cv=5,  n_jobs=-1, error_score='raise'
)

# fit the model
s = np.vstack(slow_observations_train)
slow_random_search.fit(s)
f = np.vstack(fast_observations_train)
fast_random_search.fit(f)

/opt/miniconda3/lib/python3.9/site-packages/joblib/externals/loky/process_executor.py:702: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
/opt/miniconda3/lib/python3.9/site-packages/joblib/externals/loky/process_executor.py:702: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


RandomizedSearchCV(cv=5, error_score='raise',
                   estimator=GaussianMixture(reg_covar=0.0001), n_iter=1000,
                   n_jobs=-1,
                   param_distributions={'covariance_type': ['full', 'tied',
                                                            'diag',
                                                            'spherical'],
                                        'init_params': ['kmeans', 'k-means++',
                                                        'random',
                                                        'random_from_data'],
                                        'max_iter': array([  100,   200,   300,   400,   500,   600,   700,   800,   900,
        1000,  1100,  1200,  1300,  1400,  1500,  1600,  1700,  1800,
        19...
        6400,  6500,  6600,  6700,  6800,  6900,  7000,  7100,  7200,
        7300,  7400,  7500,  7600,  7700,  7800,  7900,  8000,  8100,
        8200,  8300,  8400,  8500,  8600,  8700,  8800,  8900,  9000,
        9100,  9200,  9300,  9400,  9500,  9600,  9700,  9800,  9900,
       10000]),
                                        'n_components': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10]),
                                        'random_state': array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10]),
                                        'tol': array([0.001, 0.002, 0.003, 0.004, 0.005, 0.006, 0.007, 0.008, 0.009,
       0.01 ])})

In [18]:
slow_model = slow_random_search.best_estimator_
fast_model = fast_random_search.best_estimator_

slow_model.fit(s)
fast_model.fit(f)

GaussianMixture(init_params='random', max_iter=2800, n_components=9,
                random_state=0, reg_covar=0.0001, tol=0.007)

In [19]:
# test the HMM
accuracy = 0

for obs in slow_observations_test:
    if slow_model.score(obs) > fast_model.score(obs):
        accuracy += 1

for obs in fast_observations_test:
    if fast_model.score(obs) > slow_model.score(obs):
        accuracy += 1

accuracy = accuracy / (len(slow_observations_test) + len(fast_observations_test))
print("Accuracy: ", accuracy)

Accuracy:  0.5161290322580645


In [20]:
"""
# initialize the grid search cv
slow_grid = GridSearchCV(slow_model, hyperparametes, cv=5, n_jobs=-1, verbose=1)
fast_grid = GridSearchCV(fast_model, hyperparametes, cv=5, n_jobs=-1, verbose=1)

# fit the model
s = np.vstack(slow_observations_train)
slow_grid.fit(s)
f = np.vstack(fast_observations_train)
fast_grid.fit(f)

# get the best estimators
slow_hmm = slow_grid.best_estimator_
fast_hmm = fast_grid.best_estimator_

slow_hmm.fit(s)
fast_hmm.fit(f)

# test the HMM
accuracy = 0

for obs in slow_observations_test:
    if slow_hmm.score(obs) > fast_hmm.score(obs):
        accuracy += 1

for obs in fast_observations_test:
    if fast_hmm.score(obs) > slow_hmm.score(obs):
        accuracy += 1

accuracy = accuracy / (len(slow_observations_test) + len(fast_observations_test))
print("Accuracy: ", accuracy)
"""

'\n# initialize the grid search cv\nslow_grid = GridSearchCV(slow_model, hyperparametes, cv=5, n_jobs=-1, verbose=1)\nfast_grid = GridSearchCV(fast_model, hyperparametes, cv=5, n_jobs=-1, verbose=1)\n\n# fit the model\ns = np.vstack(slow_observations_train)\nslow_grid.fit(s)\nf = np.vstack(fast_observations_train)\nfast_grid.fit(f)\n\n# get the best estimators\nslow_hmm = slow_grid.best_estimator_\nfast_hmm = fast_grid.best_estimator_\n\nslow_hmm.fit(s)\nfast_hmm.fit(f)\n\n# test the HMM\naccuracy = 0\n\nfor obs in slow_observations_test:\n    if slow_hmm.score(obs) > fast_hmm.score(obs):\n        accuracy += 1\n\nfor obs in fast_observations_test:\n    if fast_hmm.score(obs) > slow_hmm.score(obs):\n        accuracy += 1\n\naccuracy = accuracy / (len(slow_observations_test) + len(fast_observations_test))\nprint("Accuracy: ", accuracy)\n'

Initial Accuracy Including Every Feature:
51.61%
